In [2]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import contractions

In [51]:
import pandas as pd

splits = {'train': 'train.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/BhabhaAI/DEITA-Complexity/" + splits["train"])

df.to_csv("scorer_data.csv")

print(df.head)

<bound method NDFrame.head of                                                   prompt  score
0                              Who is Jorge Luis Borges?      1
1      \nGiven a list of movies, determine which ones...      5
2      \nIdentify the neighboring major countries of ...      2
3                     What is salvation in christianity?      1
4                         What is the capital of France?      1
...                                                  ...    ...
65492  \nGiven a financial instrument known as the Se...      3
65493  \nConsidering various factors such as personal...      5
65494  \nIdentify the individual who pioneered the in...      3
65495  \nGiven a comprehensive happiness report that ...      5
65496  \nConsidering that a kids' soccer game typical...      5

[65497 rows x 2 columns]>


In [3]:
def preprocess(text):
    # Check if the input is a string
    if not isinstance(text, str):
        return ''  # Return an empty string for non-string input
    
    # Expand contractions
    text = contractions.fix(text)

    # Remove HTML tags, special characters, keep alpha numberic + punctuation
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s\-\']', ' ', text)
    
    # Lowercase, Tokenize the text
    text = text.lower()
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Combine tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [75]:
df = df[df["score"] > 0]
df = df[df["score"] < 7]

df.groupby("score").count()
df = df.dropna(how="all")
df.head

<bound method NDFrame.head of        Unnamed: 0                                             prompt  score  \
0               0                          Who is Jorge Luis Borges?      1   
1               1  \nGiven a list of movies, determine which ones...      5   
2               2  \nIdentify the neighboring major countries of ...      2   
3               3                 What is salvation in christianity?      1   
4               4                     What is the capital of France?      1   
...           ...                                                ...    ...   
65492       65492  \nGiven a financial instrument known as the Se...      3   
65493       65493  \nConsidering various factors such as personal...      5   
65494       65494  \nIdentify the individual who pioneered the in...      3   
65495       65495  \nGiven a comprehensive happiness report that ...      5   
65496       65496  \nConsidering that a kids' soccer game typical...      5   

                     

In [52]:
from tqdm import tqdm
tqdm.pandas()
df["processed_prompt"]=df["prompt"].apply(preprocess)

df.to_csv("scorer_data.csv")
# df["prompt"]

In [71]:
df = pd.read_csv("scorer_data.csv")

In [76]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Example dataset
sentences = df["processed_prompt"]
complexity_scores = df["score"]

# Preprocessing and feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)
y = np.array(complexity_scores)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
print(f'MSE: {mean_squared_error(y_test, y_pred)}')

# Function to predict complexity
def predict_complexity(sentence):
    features = vectorizer.transform([sentence])
    score = model.predict(features)
    return score[0]

# Example usage
print(predict_complexity("How complex is this sentence?"))

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [27]:
print(predict_complexity("PrerequisitesBefore we start, make sure that you have the PyMongo distribution installed. In the Python shell, the following should run without raising an exception:import pymongoThis tutorial also assumes that a MongoDB instance is running on the default host and port. Assuming you have downloaded and installed MongoDB, you can start it like so:$ mongod"))
print(predict_complexity("botta hella complex"))
print(predict_complexity("write me a really insane essay on dogs."))
print(predict_complexity("who was the 15th president usa"))
print(predict_complexity("write me a full stack project on astrophysics with visualization"))
print(predict_complexity("Create a speech about miso soup and its health benefits and why we should eat it everyday. Do it based on the guidelines I am sending you and make sure it is about 9 minutes in length. You can create the visual aid yourself. Make it about how miso soup is made.  Guidelines: Overview A speaking outline is a condensed version of a preparation outline. You use the speaking outline when you deliver your speech. It contains just enough information to jog your memory and remind you of what you want to say. It should not have so much information on it that you could read it out loud and it would make sense. Instead, it consists of sentence fragments or key words. There are two exceptions to this rule: Quotations and oral citations. If you quote someone in your speech, it should be written on the speaking outline exactly as the source stated it. The speaking outline also includes detailed oral citations.  Please see the Oral Citation Guide for additional information on what to include in an oral citation.  Divide and label the outline into the main components of a speech, Introduction, Body, and Conclusion. Include transitions between each section of the speech. The transitions should be in parenthesis. Include more detailed oral citationsInclude exact wording of quotations if using Include brief visual aid indicators. In other words, what you plan to say to introduce the visual aid, how you will explain the visual aid, and what you plan to say immediately after you show your visual aid to the audience. MAKE SURE TO GIVE ME THE VISUAL AID AND ONLY USE REAL SOURCES AND PROVIDE LINKS TO THOSE SOURCES"))

3


In [48]:
list_prompts = df[df["score"] == 6]["prompt"].to_list()

In [49]:
for i in list_prompts:
    print(f"Off by {9-predict_complexity(i)}")

Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 3
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
Off by 4
O